In [ ]:
import fig02to05 as src
import pandas as pd

pd.set_option("display.float_format", "{:.4f}".format)


id_cols = ["sub", "task", "group", "study"]
vas_cols = ["int", "unp"]
psr_cols = ["nps", "siips", "na-gen", "na-therm", "na-mech", "na-sound", "na-vis"]

# study1: blood oxygen level dependent imaging
bold = (
    pd.read_csv("../../results/bold-ps-agg.csv")
    .assign(study="bold")
    .pipe(src.scale, psr_cols)  # zscore
)

# study2: cerebral blood flow imaging
cbf = (
    pd.read_csv("../../results/cbf-ps-agg.csv")
    .assign(study="cbf")
    .query("task.str.endswith('49C') and group != 's'")  # select 49C runs, c-m-l groups
    .pipe(src.drop_outliers, "cbfQEI")  # -1.5 * IQR on CBF quality index
    .pipe(src.scale, psr_cols)  # zscore
)

# study1 + study2: pain signature responses
psr = (
    pd.concat([bold[id_cols + vas_cols + psr_cols], cbf[id_cols + vas_cols + psr_cols]])
    .groupby("sub")  # filter missing data
    .filter(lambda x: {"preManip49C", "postManip49C"}.issubset(x["task"]))
)

psr.groupby(["group", "study"])["sub"].nunique()


**Fig02: pain ratings**

In [ ]:
tasks, groups = ["preManip49C", "postManip49C", "baseline49C"], ["m", "c"]

fig02 = (
    psr.query("task.isin(@tasks) and group.isin(@groups)")
    .assign(
        task=lambda x: pd.Categorical(x["task"], tasks, ordered=True),
        group=lambda x: pd.Categorical(x["group"], groups, ordered=True),
    )
    .melt(id_vars=id_cols, value_vars=vas_cols)
)

src.plot_group_by_manipulation(
    fig02,
    var_order=vas_cols,
    task_order=["baseline49C", "preManip49C", "postManip49C"],
    group_order=["m", "c"],
)

(
    fig02.groupby("variable")
    .apply(src.fit_mixedlm, formula="value ~ group * task + study")
    .groupby(level=1)
    .apply(src.multicomp_fdr)
    .style.applymap(lambda fdr: "background-color: lightblue" if fdr < 0.05 else "", subset="qFDR")
)


**Fig03: pain signature responses**

In [ ]:
tasks, groups = ["preManip49C", "postManip49C"], ["m", "c"]

fig03 = (
    psr.query("task.isin(@tasks) and group.isin(@groups)")
    .assign(
        task=lambda x: pd.Categorical(x["task"], tasks, ordered=True),
        group=lambda x: pd.Categorical(x["group"], groups, ordered=True),
    )
    .melt(id_vars=id_cols, value_vars=psr_cols)
)

src.plot_group_by_manipulation(
    fig03,
    var_order=None,
    task_order=["preManip49C", "postManip49C"],
    group_order=["m", "c"],
)

(
    fig03.groupby("variable")
    .apply(src.fit_mixedlm, formula="value ~ group * task + study")
    .groupby(level=1)
    .apply(src.multicomp_fdr)
    .style.applymap(lambda fdr: "background-color: lightblue" if fdr < 0.05 else "", subset="qFDR")
)


**Fig05: mindfulness meditation vs placebo cream**

In [ ]:
tasks, groups = ["preManip49C", "postManip49C", "baseline49C"], ["l", "m"]

fig05a = (
    psr.query("task.isin(@tasks) and group.isin(@groups) and study == 'cbf'")
    .assign(
        task=lambda x: pd.Categorical(x["task"], tasks, ordered=True),
        group=lambda x: pd.Categorical(x["group"], groups, ordered=True),
    )
    .melt(id_vars=id_cols, value_vars=vas_cols)
)

src.plot_group_by_manipulation(
    fig05a,
    var_order=None,
    task_order=["baseline49C", "preManip49C", "postManip49C"],
    group_order=["m", "l"],
)

(
    fig05a.groupby("variable")
    .apply(src.fit_mixedlm, formula="value ~ group * task")
    .groupby(level=1)
    .apply(src.multicomp_fdr)
    .style.applymap(lambda fdr: "background-color: lightblue" if fdr < 0.05 else "", subset="qFDR")
)


In [ ]:
tasks, groups = ["preManip49C", "postManip49C"], ["l", "m"]

fig05b = (
    psr.query("task.isin(@tasks) and group.isin(@groups) and study == 'cbf'")
    .assign(
        task=lambda x: pd.Categorical(x["task"], tasks, ordered=True),
        group=lambda x: pd.Categorical(x["group"], groups, ordered=True),
    )
    .melt(id_vars=id_cols, value_vars=["nps", "siips"])
)

src.plot_group_by_manipulation(
    fig05b,
    var_order=None,
    task_order=["preManip49C", "postManip49C"],
    group_order=["m", "l"],
)

(
    fig05b.groupby("variable")
    .apply(src.fit_mixedlm, formula="value ~ group * task")
    .groupby(level=1)
    .apply(src.multicomp_fdr)
    .style.applymap(lambda fdr: "background-color: lightblue" if fdr < 0.05 else "", subset="qFDR")
)
